---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

In [1]:
# python standard library
import re
from abc import ABCMeta
from abc import abstractproperty
from collections import namedtuple

# third party
import pandas

In [2]:
COLUMN_AXIS = 1

In [3]:
class BaseData(object):
    """base class for the data-loaders

    Parameters
    ----------

    constants: object
       object with the constant values expected by the data-builder
    """
    __metaclass__ = ABCMeta

    def __init__(self, constants):
        self.constants = constants
        self._data = None
        return


    def rename_countries(self, country):
        """renames country

        .. note:: Expects the constants to have a dict named `country_remap`

        Parameters
        ----------

        country (str):
           name of the country

        Returns
        -------

        str: country to use in the column
        """
        return self.constants.country_remap.get(country, country)

    @abstractproperty
    def data(self):
        """the data-frame"""
        return

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [4]:
class EnergyIndicatorsConstants:
    source = "Energy Indicators.xls"
    columns = ['Country', 'Energy Supply', 'Energy Supply per Capita',
               '% Renewable']
    header_rows = 17
    footer_rows = 38
    remove_columns_count = 2
    missing_values = "..."
    country_remap = {"Republic of Korea": "South Korea",
                     "United States of America20": "United States",
                     "United Kingdom of Great Britain and Northern Ireland19": "United Kingdom",
                     "China, Hong Kong Special Administrative Region3": "Hong Kong",
                     "China, Macao Special Administrative Region4": "Macao"}
    # this will wipe out Hong Kong so use the regex after the remap
    country_key = "country"
    country_regex = re.compile(
        "(?P<{0}>[a-zA-Z\s]+)\b?(\(|\d)*".format(country_key))
    
class EnergyIndicatorsKeys(object):
    """Keys for the data-frame"""
    country_name = "Country"
    per_capita = "Energy Supply per Capita"
    percent_renewable = "% Renewable"
    population = "population"
    total_supply = "Energy Supply"

class UnitConverter(object):
    petajoule_to_gigajoule = 10**6
    

In [5]:
class EnergyIndicators(BaseData):
    """builds the energy indicator data-frame

    Parameters
    ----------

    constants (objects): object with the string constants
    """
    def __init__(self, *args, **kwargs):
        super(EnergyIndicators, self).__init__(*args, **kwargs)
        self._to_remove = None
        return

    @property
    def to_remove(self):
        """leading columns to remove

        Returns
        -------

        list: names to use for columns to remove from the beginning of sheet
        """
        if self._to_remove is None:
            self._to_remove = [
                "Bad_{0}".format(index)
                for index in range(self.constants.remove_columns_count)]
        return self._to_remove

    def clean_countries(self, country):
        """Gets rid of the extra characters put in by the excel spreadsheet

        Parameters
        ----------

        country: string
            country name to clean up

        Returns
        -------

        str: country name with extra characters removed
        """
        return self.constants.country_regex.search(country).group(
            self.constants.country_key)


    @property
    def data(self):
        """DataFrame with energy indicators"""
        if self._data is None:
            self._data = pandas.read_excel(self.constants.source,
                                           names=self.to_remove + self.constants.columns,
                                           skiprows=self.constants.header_rows,
                                           skip_footer=self.constants.footer_rows,
                                           na_values=self.constants.missing_values)

            # get rid of the first two columns
            self._data = self._data.drop(self.to_remove, axis=1)
            self._data["Energy Supply"] *= UnitConverter.petajoule_to_gigajoule
            self._data.Country = self._data.Country.map(self.rename_countries)
            self._data.Country = self._data.Country.map(self.clean_countries)
            # Iran is ending up with trailing spaces
            self._data.Country = self._data.Country.str.strip()
        return self._data

In [6]:
 class GDPConstants(object):
     """holds values for the GDP data"""
     source = "world_bank.csv"
     header_rows = 4
     # the columns are based on Question one, not the data description
     columns = (["Country Name"] +
                [str(year) for year in range(2006, 2016)])
     country_remap = {"Hong Kong SAR, China": "Hong Kong",
                      "Iran, Islamic Rep.": "Iran",
                      "Korea, Rep.": "South Korea"}

In [7]:
class GDPKeys(object):
    """column names for the GDP data"""
    country_name = "Country Name"
    last_ten_years = [str(year) for year in range(2006, 2016)]
    first_year = last_ten_years[0]
    last_year = last_ten_years[-1]

In [8]:
 class GDPData(BaseData):
     """loads and cleans the gdp data

     Parameters
     ----------

     constants : object
        object with the string constants
     """
     def __init__(self, *args, **kwargs):
         super(GDPData, self).__init__(*args, **kwargs)
         return

     @property
     def data(self):
         """the GDP data

         Returns
         -------

         DataFrame: cleaned GPD data
         """
         if self._data is None:
             self._data = pandas.read_csv(self.constants.source,
                                          skiprows=self.constants.header_rows,
                                          usecols=self.constants.columns)
             self._data[GDPKeys.country_name] = self._data[GDPKeys.country_name].map(
                 self.rename_countries)
         return self._data

In [9]:
class SciamgoConstants(object):
    """strings and such to load and clean the Sciamgo data"""
    source = "scimagojr-3.xlsx"

class SciamgoKeys(object):
     """Keys for the data-frame"""
     country_name = 'Country'
     self_citations = "Self-citations"
     total_citations = "Citations"
     self_to_total_citations = "self_to_total_citations"
     citable_documents = "Citable documents"
     citable_documents_per_capita = "citable_documents_per_capti"

class SciamgoData(BaseData):
    """loads the Sciamgo Journal Country Rank data"""
    def __init__(self, *args, **kwargs):
        super(SciamgoData, self).__init__(*args, **kwargs)
        self._top_15 = None
        return

    @property
    def data(self):
        """the Sciamgo data

        Returns
        -------

        DataFrame: the country rank data
        """
        if self._data is None:
            self._data = pandas.read_excel(self.constants.source)
        return self._data

    @property
    def top_15(self):
        """Returns the top 15 entries by rank

        This is needed because question two wants you to compare how much you 
        have after you merge without the slice

        Returns
        -------

        DataFrame: slice of top 15 countries by rank
        """
        if self._top_15 is None:
            self._top_15 = self.data[self.data.Rank < 16]
        return self._top_15

In [10]:
DataFrames = namedtuple("DataFrames", "energy gdp sciamgo".split())

In [11]:
def build_data_frames():
     """builds the three data-frames

     Returns
     -------

     DataFrames: named tuple with the three data-frame builders

      ,* energy: EnergyIndicators
      ,* gdp: GDPData
      ,* sciamgo: SciamgoData
     """
     energy_indicators = EnergyIndicators(EnergyIndicatorsConstants)
     gdp = GDPData(GDPConstants)
     sciamgo = SciamgoData(SciamgoConstants)
     return DataFrames(energy=energy_indicators, gdp=gdp, sciamgo=sciamgo)


In [12]:
def answer_one():
    """loads the data, cleans and combines them

    - Energy Indicators
    - GDP Data
    - Sciamgo Journal data

    Returns
    -------

    DataFrame : compiled (20 x 15) data
    """
    frames = build_data_frames()
    merged = pandas.merge(frames.sciamgo.top_15, frames.energy.data, how="left",
                          on=[SciamgoKeys.country_name,
                              EnergyIndicatorsKeys.country_name])
    merged = pandas.merge(merged, frames.gdp.data, how='left', left_on=SciamgoKeys.country_name,
                          right_on=GDPKeys.country_name)
    merged = merged.set_index(merged[SciamgoKeys.country_name])
    return merged.drop(labels=[SciamgoKeys.country_name, GDPKeys.country_name], axis=COLUMN_AXIS)

merged = answer_one()
expected_columns = ['Rank', 'Documents', 'Citable documents', 
                    'Citations', 'Self-citations',
                    'Citations per document', 'H index', 'Energy Supply',
                    'Energy Supply per Capita', '% Renewable', '2006',
                    '2007', '2008', '2009', '2010', '2011', '2012',
                    '2013', '2014', '2015']
for index, column in enumerate(merged.columns):
    assert column == expected_columns[index]
assert merged.shape == (15, 20), merged.shape

FileNotFoundError: [Errno 2] No such file or directory: 'Energy Indicators.xls'

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [13]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [ ]:
def answer_two():
    """calculates difference between union and intersections

    Returns
    -------

    int: data lost by joining
    """
    frames = build_data_frames()
    union = pandas.merge(frames.sciamgo.data, frames.energy.data, how="outer",
                          on=[SciamgoKeys.country_name,
                          EnergyIndicatorsKeys.country_name])
    union = pandas.merge(union, frames.gdp.data, how='outer',
                         left_on=SciamgoKeys.country_name,
                         right_on=GDPKeys.country_name)
    intersection = pandas.merge(frames.sciamgo.data, frames.energy.data, how="left",
                                on=[SciamgoKeys.country_name,
                                    EnergyIndicatorsKeys.country_name])
    intersection = pandas.merge(intersection, frames.gdp.data, how='left',
                                left_on=SciamgoKeys.country_name,
                                right_on=GDPKeys.country_name)
    

    union_count = len(union)
    intersection_count = len(intersection)
    return union_count - intersection_count

# print(answer_two())

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [ ]:
def answer_three():
    """calculates average gdp

    Returns
    -------

    Series: mean GDP for each countr
    """
    Top15 = answer_one()
    return Top15[GDPKeys.last_ten_years].mean(axis=1)

# print(tabulate(pandas.DataFrame(answer_three()),
#                headers=["Country", "Average GDP (2009 to 2015)"],
#                tablefmt="orgtbl"))

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [ ]:
def answer_four():
    """Calculates GDP change over ten year span for country with 6th largest
    Returns
    -------

    float: change in GDP from 2006 to 20015 for 6th largest GDP in top 15
    """
    top_15 = answer_one()
    means = top_15[GDPKeys.last_ten_years].mean(axis=1)
    index = means[means==means.nlargest(6)[-1]].index
    sixth = top_15.loc[index]
    return (sixth[GDPKeys.last_year] - sixth[GDPKeys.first_year]).values[0]

# print(str(answer_four()))

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [ ]:
def answer_five():
    """mean Energy Supply per Capita"""
    Top15 = answer_one()
    return Top15[EnergyIndicatorsKeys.per_capita].mean()
# print(answer_five())

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [ ]:
def answer_six():
    """finds country with most renewable energy

    Returns
    -------

    tuple: (country name, percentage renewable)
    """
    Top15 = answer_one()
    country = Top15[EnergyIndicatorsKeys.percent_renewable].argmax()    
    amount = Top15.loc[country][EnergyIndicatorsKeys.percent_renewable]
    return (country, amount)

# print(answer_six())


### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():
    """adds Self-Citations/Total Citations column

    Returns
    -------

    tuple: (country with highest ratio, highest ratio)
    """
    Top15 = answer_one()
    ratio_key = SciamgoKeys.self_to_total_citations
    self_key = SciamgoKeys.self_citations
    Top15[ratio_key] = Top15[self_key]/Top15[SciamgoKeys.total_citations]
    country = Top15[ratio_key].argmax()
    value = Top15.loc[country][ratio_key]
    return (country, value)

# print(answer_seven())

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [ ]:
def add_population(data):
    """adds population estimate

    Parameters
    ----------

    data: DataFrame
       data with energy-per-capita and total energy columns

    Returns
    -------

    DataFrame: copy of data with `population` column
    """
    population = EnergyIndicatorsKeys.population
    per_capita = EnergyIndicatorsKeys.per_capita
    total = EnergyIndicatorsKeys.total_supply
    data[population] = data[total]/data[per_capita]
    return data

In [ ]:
def answer_eight():
    """adds population estimate column
    
    Returns
    -------
    str: name of third most populous country
    """
    top_15 = add_population(answer_one())
    third = top_15.population.nlargest(3)[-1]
    return top_15[top_15.population == third].index[0]

# print(answer_eight())

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    """adds citable documents per person column

    Calculates the correlation between citations/capita and energy supply/capita
    Returns
    -------

    float: correlation
    """
    ratio_key = SciamgoKeys.citable_documents_per_capita
    citeable_documents = SciamgoKeys.citable_documents

    per_capita = EnergyIndicatorsKeys.per_capita
    top_15 = add_population(answer_one())
    top_15[ratio_key] = top_15[citeable_documents]/top_15.population
    return top_15[[ratio_key, per_capita]].corr().values[0, -1]

# print(answer_nine())

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    """classifies by median % renewable

    Returns
    -------

    Series: 1 if at or above median, 0 otherwaise
    """
    renewable_key = EnergyIndicatorsKeys.percent_renewable
    Top15 = answer_one()
    median = Top15[renewable_key].median()

    def classify(value):
        return 1 if value >= median else 0
    return Top15[renewable_key].map(classify)

# print(tabulate(pandas.DataFrame(answer_ten()),
#                headers=["Country", "Meets Median Renewable"],
#                tablefmt="orgtbl"))

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

In [ ]:
def answer_eleven():
    """Groups countries by continent
    
    * displays countries per continent
    * displays sum, mean, std for estimated population of each country

    Returns
    -------

    DataFrame:
       index uses contintes, columns or size, sum, mean, and std
    """
    top_15 = add_population(answer_one())
    top_15["Continent"] = top_15.index.map(lambda x: ContinentDict[x])
    group = top_15.groupby("Continent")
    return pandas.DataFrame({"size": group.count()["2009"],
                             "sum": group.population.sum(),
                             "mean": group.population.mean(),
                             "std": group.population.std()})
outcome = answer_eleven()
# print(tabulate(outcome, headers="keys", tablefmt="orgtbl"))

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    """bins percent renewable

    groups top 15 by continent as well as 5 renewable bins

    Returns
    -------

    Series: with multiindex using bins for percent renewable and continent
    """
    Top15 = answer_one()
    Top15["Continent"] = Top15.index.map(lambda x: ContinentDict[x])
    return Top15.groupby(["Continent",
                          pandas.cut(Top15[
                              EnergyIndicatorsKeys.percent_renewable], 5,
                          labels=["bin{0}".format(bin) for bin in range(5)])])[GDPKeys.last_year].count()


# print(answer_twelve())

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    """Converts population to a formatted string"""
    Top15 = add_population(answer_one())
    return Top15.population.map(lambda x: "{0:,}".format(x))

output = answer_thirteen()
# print(tabulate(pandas.DataFrame(output), headers="keys", tablefmt="orgtbl"))

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
# plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!